<a href="https://colab.research.google.com/github/gabiamorim2/projeto_covid-19/blob/main/projeto_COVID_19_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Análise exploratória e visualização interativa de dados: COVID-19**

## **1\. Contexto**



A **COVID-19** é uma infecção respiratória aguda causada pelo coronavírus **SARS-CoV-2**, potencialmente grave, de elevada transmissibilidade e de distribuição global (Fonte: Ministério da Saúde - Governo Federal -  [Link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus))


A disponibilidade de dados sobre a evolução da pandemia em um intervalo de tempo e em uma determinada região geográfica do país e/ou mundo, é fundamental para o **desenvolvimento de estratégias para seu combate e tratamento**. Esse projeto tem como objetivo a construção de um **dashboard de dados para exploração e visualização interativa** sobre o avanço de casos e da vacinação no Brasil. 

###**1.2 Resumo**

*   **Dashboard:**
 - Google Data Studio ([`link`](https://datastudio.google.com/reporting/9c6322b8-b5cb-4f91-a149-22f248fa50b1));

*   **Processamento dos dados:**
 - Kaggle ([`link`](https://www.kaggle.com/code/gabrielaamorim/projeto-covid-19#));

*   **Fonte das informações:**
 - Base de dados sobre casos de Covid-19 - **Universidade John Hopkins** ([`link`](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
 - Base de dados sobre vacinação no Brasil - **Universidade de Oxford** ([`link`](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

###**1.3 Sobre os dados**

Os dados sobre **casos** de COVID-19 são compilados pelo **Centro de Ciência de Sistemas e Engenharia** da universidade norte-americana **John Hopkins**. As informações são atualizadas diariamente desde janeiro de 2020 com uma granularidade temporal de dias e granularidade geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [`link`](https://systems.jhu.edu/research/public-health/ncov/), e os dados completos neste [`link`](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).

Abaixo estão as informações detalhadas derivadas do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país; 
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número de infectados nas últimas 24 horas;
 - **confirmed_moving_avg_7d**: média móvel do número de infectados nos últimos 7 dias;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortes;
 - **deaths_1d**: número de mortes nas últimas 24 horas;
 - **deaths_moving_avg_7d**: média móvel do número de mortes nos últimos 7 dias;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre **vacinação** foram compilados pelo projeto **Nosso Mundo em Dados** (***Our World in Data - OWID***) da universidade britânica de **Oxford** ([`link`](https://www.ox.ac.uk)). As informações são atualizadas diariamente desde janeiro de 2020 com uma granularidade temporal de dias e granularidade geográfica de países. O site do projeto pode ser acessado nesse [`link`](https://ourworldindata.org), enquanto que os dados completos podem ser acessados neste [`link`](https://covid.ourworldindata.org/data/owid-covid-data.csv).


Abaixo estão as informações detalhadas derivadas do seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número total de doses administradas;
 - **one_shot**: número total de pessoas que tomaram uma dose;
 - **one_shot_perc**: número percentual relativo de pessoas com uma dose;
 - **two_shots**: número total de pessoas que tomaram duas doses;
 - **two_shot_perc**: número percentual relativo de pessoas com duas doses;
 - **three_shots**: número total de pessoas que tomaram três doses;
 - **three_shot_perc**: número percentual relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## **2\. Análise Exploratória de Dados**

Nesta seção, vamos processar os dados brutos de forma que fiquem adequados para a construção de um dashboard interativo ao final do projeto. Para as análises, vamos utilizar os seguintes pacotes:

In [ ]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## **2\.1 Casos de infectados**



###**2.1.1 Extração**

Vamos começar processando as informações de **casos** de COVID-19 provenientes da base de dados da universidade John Hopkins. Os dados foram compilados em um arquivo por dia (por exemplo: 2021/12/01).

In [ ]:
casos = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [ ]:
casos.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


Vamos definir uma função para iterar dentro de um intervalo de tempo definido e extrair as informações que queremos.

In [ ]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [ ]:
#definindo o intervalo de tempo

start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

De maneira iterativa, vamos selecionar as colunas de interesse e as informações referentes ao Brasil.

In [ ]:
casos = None
casos_vazio = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  caso = pd.read_csv(data_source_url, sep=',')

  caso = caso.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  caso = caso.query('Country_Region == "Brazil"').reset_index(drop=True)
  caso['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if casos_vazio:
    casos = caso
    casos_vazio = False
  else:
    casos = casos.append(caso, ignore_index=True)

###**2.1.2 Transformação**

Vamos manejar as informações de forma a garantir uma boa granularidade e qualidade dos dados.

In [ ]:
casos.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [ ]:
#quantidade de linhas e colunas do dataframe

casos.shape

(9828, 6)

In [ ]:
#informações sobre os tipos de dados das colunas

casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


Vamos começar ajustando o **nome das colunas** de interesse e deixá-los todos com letra minúscula.

In [ ]:
casos = casos.rename(columns={
    'Province_State': 'state',
    'Country_Region': 'country'
})

for col in casos.columns:
  casos = casos.rename(columns={col: col.lower()})

Editando o nome dos **estados**.

In [ ]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

casos['state'] = casos['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Criação duas novas colunas com **chaves temporais**.


In [ ]:
#criação das colunas 'month' e 'year'

casos['month'] = casos['date'].apply(lambda date: date.strftime('%Y-%m'))
casos['year'] = casos['date'].apply(lambda date: date.strftime('%Y'))

Criação de uma nova coluna com **população estimada de cada estado**.

In [ ]:
casos['population'] = round(100000 * (casos['confirmed'] / casos['incident_rate']))
casos = casos.drop('incident_rate', axis=1)

Criação de colunas com **número de mortes e casos nas últimas 24 horas, média móvel de mortes e casos em 7 dias e estabilidade (14 dias)**, por estado.

In [ ]:
#criação das colunas confirmed_1d, confirmed_moving_avg_7d, confirmed_moving_avg_7d_rate_14d, confirmed_trend, deaths_1d, deaths_moving_avg_7d, deaths_moving_avg_7d_rate_14d,
# e deaths_trend


casos_ = None
casos_vazio = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'em baixa'
  elif rate > 1.15:
    status = 'em alta'
  else:
    status = 'estável'

  return status

for state in casos['state'].drop_duplicates():

  cases_per_state = casos.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d'] / cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d'] / cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if casos_vazio:
    casos_ = cases_per_state
    casos_vazio = False
  else:
    cases_ = casos_.append(cases_per_state, ignore_index=True)

casos = cases_
cases_ = None

**Type casting das colunas:** vamos garantir que as novas colunas estejam com o tipo certo de dado para que não exista nenhum erro na hora da manipulação das informações.

In [ ]:
casos['population'] = casos['population'].astype('Int64')
casos['confirmed_1d'] = casos['confirmed_1d'].astype('Int64')
casos['confirmed_moving_avg_7d'] = casos['confirmed_moving_avg_7d'].astype('Int64')
casos['deaths_1d'] = casos['deaths_1d'].astype('Int64')
casos['deaths_moving_avg_7d'] = casos['deaths_moving_avg_7d'].astype('Int64')

Por fim, vamos estabelecer a **ordem das colunas** para facilitar a visualização das informações no dataframe finalizado.

In [ ]:
casos = casos[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d',
               'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [ ]:
casos.head(25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


###**2.1.3 Carregando os dados**

Com os dados de casos de COVID-19 editados e prontos, vamos gravá-los em disco como `arquivo csv` e posteriormente fazer o download para carregá-lo no **Google Data Studio**.

In [ ]:
casos.to_csv('./covid-casos.csv', sep=',', index=False)

## **2\.2 Vacinação no Brasil** 


Vamos seguir processando os dados, que agora são sobre **vacinação contra COVID-19 no Brasil**, as informações são provenientes de um banco de dados da universidade de Oxford. Ao contrário do outro dataset, os dados estão compilados em um arquivo.

###**2.2.1 Extração**

In [ ]:
#leitura do arquivo

vacinas = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

In [ ]:
vacinas.head()

Vamos selecionar as **colunas de interesse** e as informações referentes ao **Brasil**.

In [ ]:
vacinas = vacinas.query('location == "Brazil"').reset_index(drop=True)
vacinas = vacinas[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

###**2.2.2 Transformação**

Vamos manejar as informações de forma a garantir uma boa granularidade e qualidade dos dados.

In [ ]:
vacinas.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,213993441.0,NaN,NaN,NaN,NaN,2020-02-26
1,Brazil,213993441.0,NaN,NaN,NaN,NaN,2020-02-27
2,Brazil,213993441.0,NaN,NaN,NaN,NaN,2020-02-28
3,Brazil,213993441.0,NaN,NaN,NaN,NaN,2020-02-29
4,Brazil,213993441.0,NaN,NaN,NaN,NaN,2020-03-01


In [ ]:
#conferindo o número de linhas e colunas do dataframe

vacinas.shape

(849, 7)

In [ ]:
#verificando as informações sobre o dataset

vacinas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 849 non-null    object        
 1   population               849 non-null    float64       
 2   total_vaccinations       503 non-null    float64       
 3   people_vaccinated        499 non-null    float64       
 4   people_fully_vaccinated  483 non-null    float64       
 5   total_boosters           275 non-null    float64       
 6   date                     849 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 46.6+ KB


Como observado acima, existem **dados nulos** no dataframe, portanto vamos começar tratando os valores ausentes substituindo-os com o valor anterior válido mais proximo.

In [ ]:
#utilizando o método ffill para preencher os valores ausentes

vacinas = vacinas.fillna(method='ffill')

Vamos filtrar a base de dados de acordo com a coluna **`date`**, para que a granularidade dos dados seja a mesma nos dois arquivos.

In [ ]:
vacinas = vacinas[(vacinas['date'] >= '2021-01-01') & (vacinas['date'] <= '2021-12-31')].reset_index(drop=True)

Vamos alterar o nome de algumas colunas.

In [ ]:
vacinas = vacinas.rename(columns={
    'location':'country',
    'total_vaccinations':'total',
    'people_vaccinated':'one_shot',
    'people_fully_vaccinated':'two_shots',
    'total_boosters':'three_shots',
    
 }
)

Criação de duas novas colunas com **chaves temporais**.

In [ ]:
#criação das colunas 'month' e 'year'

vacinas['month'] = vacinas['date'].apply(lambda date: date.strftime('%Y-%m'))
vacinas['year'] = vacinas['date'].apply(lambda date: date.strftime('%Y'))

Criação de três novas colunas com **percentual da população vacinada** com cada dose.

In [ ]:
vacinas['one_shot_perc'] = round(vacinas['one_shot'] / vacinas['population'], 4)
vacinas['two_shots_perc'] = round(vacinas['two_shots'] / vacinas['population'], 4)
vacinas['three_shots_perc'] = round(vacinas['three_shots'] / vacinas['population'], 4)

**Type casting das colunas:** vamos garantir que as novas colunas estejam com o tipo certo de dado para que não exista nenhum erro na hora da manipulação das informações.

In [ ]:
vacinas['population'] = vacinas['population'].astype('Int64')
vacinas['total'] = vacinas['total'].astype('Int64')
vacinas['one_shot'] = vacinas['one_shot'].astype('Int64')
vacinas['two_shots'] = vacinas['two_shots'].astype('Int64')
vacinas['three_shots'] = vacinas['three_shots'].astype('Int64')

Por fim, vamos estabelecer a **ordem das colunas** para facilitar a visualização das informações no dataframe finalizado.

In [ ]:
vacinas = vacinas[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc',
                   'month', 'year']]

In [ ]:
vacinas.head()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
0,2021-01-01,Brazil,213993441,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
1,2021-01-02,Brazil,213993441,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
2,2021-01-03,Brazil,213993441,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
3,2021-01-04,Brazil,213993441,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
4,2021-01-05,Brazil,213993441,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021


In [ ]:
vacinas.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
360,2021-12-27,Brazil,213993441,329011365,165952037,0.7755,142764283,0.6671,25218893,0.1178,2021-12,2021
361,2021-12-28,Brazil,213993441,329861730,166062249,0.7760,142965728,0.6681,25758909,0.1204,2021-12,2021
362,2021-12-29,Brazil,213993441,330718457,166143380,0.7764,143282084,0.6696,26219623,0.1225,2021-12,2021
363,2021-12-30,Brazil,213993441,331164041,166185628,0.7766,143398692,0.6701,26507937,0.1239,2021-12,2021
364,2021-12-31,Brazil,213993441,331273910,166195505,0.7766,143436012,0.6703,26571077,0.1242,2021-12,2021


###**2.2.3 Carregando os dados**

Com os dados de vacinação contra COVID-19 no Brasil editados e prontos, vamos gravá-los em disco como `arquivo csv` e posteriormente fazer o download para carregá-lo no **Google Data Studio**.

In [ ]:
vacinas.to_csv('./covid-vacinas.csv', sep=',', index=False)

## **3\. Exploração Interativa de Dados**


Com as bases de dados de casos de infecção por COVID-19 e sobre vacinação no Brasil editados e prontos, vamos seguir para a construção de uma **visualização interativa** através de um **dashboard** na plataforma **`Google Data Studio.`**

###**3.1 KPIs (*key performance indicator*)** 


O dashboard contruído contém os seguintes **indicadores chaves de desempenho** **`(KPIs)`** consolidados:

**1.** Casos e mortes em 24 horas;

**2.** Média móvel de casos e mortes (em 7 dias);

**3.** Tendência de casos e mortes;

**4.** Proporção de vacinados com 1ª, 2ª e 3ª doses.

###**3.2 EDA (*exploratory data analysis*)** 


O dashboard construído contém gráficos para análise exploratória de dados interativa, são eles:

**1.** Distribuição do números de casos e mortes ao longo do tempo;

**2.** Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;

**3.** Distribuição geográfica dos casos por estado por dia (mapa).

## **4\. Dashboard**


Para acessar o dashboard, clique [aqui](https://datastudio.google.com/reporting/9c6322b8-b5cb-4f91-a149-22f248fa50b1).



---

